In [1]:
from sqlalchemy import create_engine
from db_config import *

# utilities
import re
import pickle
import numpy as np
import pandas as pd

# plotting
import seaborn as sns
import matplotlib.pyplot as plt

# nltk
from nltk.stem import WordNetLemmatizer

# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import plot_roc_curve


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report


MNB = pickle.load(open('/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Classification/Code/saved_models/Sentiment-MNB.pickle', 'rb'))
LR = pickle.load(open('/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Classification/Code/saved_models/Sentiment-LR.pickle', 'rb'))
SVM = pickle.load(open('/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Classification/Code/saved_models/Sentiment-SVC.pickle', 'rb'))
vectorisor = pickle.load(open('/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Classification/Code/saved_models/vectoriser-ngram-(1,2).pickle', 'rb'))

In [2]:
# Defining dictionary containing all emojis with their meanings.
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}
          
## Defining set containing all stopwords in english.
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from', 
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're',
             's', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [3]:
def preprocess(textdata):
    processedText = []
    
    # Create Lemmatizer and Stemmer.
    wordLemm = WordNetLemmatizer()
    
    # Defining regex patterns.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern       = '@[^\s]+'
    alphaPattern      = "[^a-zA-Z0-9]"
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    
    for tweet in textdata:
        tweet = tweet.lower()
        
        # Replace all URls with 'URL'
        tweet = re.sub(urlPattern,' URL',tweet)
        # Replace all emojis.
        for emoji in emojis.keys():
            tweet = tweet.replace(emoji, "EMOJI" + emojis[emoji])        
        # Replace @USERNAME to 'USER'.
        tweet = re.sub(userPattern,' USER', tweet)        
        # Replace all non alphabets.
        tweet = re.sub(alphaPattern, " ", tweet)
        # Replace 3 or more consecutive letters by 2 letter.
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

        tweetwords = ''
        for word in tweet.split():
            # Checking if the word is a stopword.
            #if word not in stopwordlist:
            if len(word)>1:
                # Lemmatizing the word.
                word = wordLemm.lemmatize(word)
                tweetwords += (word+' ')
            
        processedText.append(tweetwords)
        
    return processedText

In [4]:
table = 'social.machine_learning_only'
classifier_name = 'MNB'
start_date = '2021-01-01'
end_date = '2021-12-31'


engine = create_engine('mysql+mysqlconnector://'+ user + ':' + passwd + '@' + ip + ':3306/' + schema1)
df = pd.read_sql(f'SELECT * FROM {table} ', engine)

In [5]:
import time
text = df['text']
t = time.time()
processedtext = preprocess(text)
print(f'Text Preprocessing complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

Text Preprocessing complete.
Time Taken: 298 seconds


In [6]:
X  = vectorisor.transform(processedtext)

In [7]:
MNB_pred = MNB.predict(X)
MNB_prob = MNB.predict_proba(X)

LR_pred = LR.predict(X)
LR_prob = LR.predict_proba(X)

SVM_pred = SVM.predict(X)
SVM_prob = SVM.predict_proba(X)

In [8]:
a = LR_pred
b = SVM_pred
c = MNB_pred

d = LR_prob
e = SVM_prob
f = MNB_prob

ensemble_pred = []
ensemble_prob = []

for i in range(len(a)):
    temp = []
    temp2 = []

    temp.append(a[i])
    temp.append(b[i])
    temp.append(c[i])
    ensemble_pred.append(max(set(temp), key=temp.count))

    temp2.append(max(d[i]))
    temp2.append(max(e[i]))
    temp2.append(max(f[i]))
    ensemble_prob.append(np.mean(temp2))

In [9]:
mnb_model_prob = []
for i in range(len(MNB_pred)):
    if MNB_pred[i] == 1:
        mnb_model_prob.append(max(MNB_prob[i]))
    else:
        mnb_model_prob.append(1 - max(MNB_prob[i]))


lr_model_prob = []
for i in range(len(LR_pred)):
    if LR_pred[i] == 1:
        lr_model_prob.append(max(LR_prob[i]))
    else:
        lr_model_prob.append(1 - max(LR_prob[i]))


svm_model_prob = []
for i in range(len(SVM_pred)):
    if SVM_pred[i] == 1:
        svm_model_prob.append(max(SVM_prob[i]))
    else:
        svm_model_prob.append(1 - max(SVM_prob[i]))


In [10]:
ML_sentiment = df.iloc[:,[0]]
ML_sentiment['mnb_prediction'] = MNB_pred
ML_sentiment['mnb_probability'] = np.array(mnb_model_prob)

ML_sentiment['lr_prediction'] = LR_pred
ML_sentiment['lr_probability'] = np.array(lr_model_prob)


ML_sentiment['svm_prediction'] = SVM_pred
ML_sentiment['svm_probability'] = np.array(svm_model_prob)


ML_sentiment['ensemble_prediction'] = np.array(ensemble_pred)
ML_sentiment['ensemble_probability'] = np.array(ensemble_prob)

<ipython-input-10-77a34c1952f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ML_sentiment['mnb_prediction'] = MNB_pred
<ipython-input-10-77a34c1952f7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ML_sentiment['mnb_probability'] = np.array(mnb_model_prob)
<ipython-input-10-77a34c1952f7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [11]:
ML_sentiment

,id,mnb_prediction,mnb_probability,lr_prediction,lr_probability,svm_prediction,svm_probability,ensemble_prediction,ensemble_probability
0,8424527990,1,0.627705,1,0.622900,1,0.583350,1,0.611319
1,8419386549,1,0.571180,0,0.480519,1,0.583250,1,0.557970
2,8415166607,1,0.521219,0,0.404371,1,0.574160,1,0.563670
3,8412898532,1,0.525191,1,0.844225,1,0.518753,1,0.629390
4,8409972932,1,0.570672,1,0.546443,1,0.573942,1,0.563686
...,...,...,...,...,...,...,...,...,...
4571780,1465833406126768128,1,0.568330,1,0.610359,1,0.583830,1,0.587506
4571781,1465833197816717316,1,0.587038,1,0.757331,1,0.583830,1,0.642733
4571782,1465833117265149963,1,0.578277,1,0.718496,1,0.583830,1,0.626868
4571783,1465833058045775878,1,0.687087,1,0.678534,1,0.571026,1,0.645549


In [12]:
ML_sentiment.to_sql('ML_sentiment' , con=engine, if_exists='append', index=False, chunksize=20000)